In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
plt.rcParams['font.family'] = 'AppleGothic'

In [2]:
ncd = pd.read_csv('/Users/user/Documents/진행중 업무/ncd_metering.csv')

In [3]:
def preprocess2 (df) :
    domain_min_dates = df.groupby(['member_id'])['calc_ymd'].min().reset_index()
    domain_min_dates = domain_min_dates.rename(columns={'calc_ymd': 'domainRegistdate'})
    df = df.merge(domain_min_dates, on=['member_id'], how='left')

    df['paid_여부'] = np.where(df['요금']>0, 'Y','N')
    return df

ncd = preprocess2(ncd)

In [4]:
ncd['plan_prefix'] = ncd['plan'].str[:3]

def preprocess3 (df):
    conversion_row = df[df['paid_여부'] == 'Y'].sort_values('calc_ymd').head(1)
    
    if not conversion_row.empty:
        conversion_date = conversion_row.iloc[0]['calc_ymd']
        df['전환여부'] = 'Y'
        df['전환일'] = conversion_date
    else:
        df['전환여부'] = 'N'
        df['전환일'] = None
    
    return df

ncd = ncd.groupby(['member_id', 'plan_prefix'], group_keys=False).apply(preprocess3)

/var/folders/yj/y3b7nzyn7zv0l9bkqtj_37g40000gn/T/ipykernel_1418/474318808.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ncd = ncd.groupby(['member_id', 'plan_prefix'], group_keys=False).apply(preprocess3)


In [5]:
ncd1 = ncd[~ncd['plan'].isin(['ATT_STD', 'FIN_STD', 'PAY_STD'])]
ncd1 = ncd1.sort_values(by = ['member_id','calc_ymd','plan_prefix','paid_여부'])
ncd1_reduced = ncd1[['member_id', 'calc_ymd', 'plan', 'plan_prefix', 'paid_여부']].drop_duplicates(subset=['member_id', 'calc_ymd', 'plan_prefix'], keep = 'last')
paid_user_plans = ncd1_reduced[ncd1_reduced['paid_여부'] == 'Y'].groupby(['member_id', 'calc_ymd'])['plan'].apply(lambda x: ', '.join(x)).reset_index(name='도메인유료구독상품')
ncd = pd.merge(ncd, paid_user_plans, on=['member_id', 'calc_ymd'], how='left')
paid_user_counts = ncd1_reduced[ncd1_reduced['paid_여부'] == 'Y'].groupby(['member_id', 'calc_ymd']).size().reset_index(name='도메인유료구독상품개수')
ncd = pd.merge(ncd, paid_user_counts, on=['member_id', 'calc_ymd'], how='left')
ncd['도메인유료구독상품개수'] = ncd['도메인유료구독상품개수'].fillna(0)
ncd = ncd.drop(['plan_prefix'], axis=1)

In [6]:
ncd['plan_prefix'] = ncd['plan'].str[:3]
ncd2 = ncd[~ncd['plan'].isin(['ATT_T', 'FIN_T', 'PAY_T', 'ATT_STD', 'FIN_STD', 'PAY_STD'])]
ncd2 = ncd2.sort_values(by = ['member_id','calc_ymd','plan_prefix','paid_여부'])
ncd2_reduced = ncd2[['member_id', 'calc_ymd', 'plan', 'plan_prefix', 'paid_여부']].drop_duplicates(subset=['member_id', 'calc_ymd', 'plan_prefix'], keep = 'last')
user_plans = ncd2_reduced.groupby(['member_id', 'calc_ymd'])['plan'].apply(lambda x: ', '.join(x)).reset_index(name='도메인구독상품')
ncd = pd.merge(ncd, user_plans, on=['member_id', 'calc_ymd'], how='left')
user_counts = ncd2_reduced.groupby(['member_id', 'calc_ymd']).size().reset_index(name='도메인구독상품개수')
ncd = pd.merge(ncd, user_counts, on=['member_id', 'calc_ymd'], how='left')
ncd['도메인구독상품개수'] = ncd['도메인구독상품개수'].fillna(0)
ncd = ncd.drop(['plan_prefix'], axis=1)

In [8]:
ncd['plan_prefix'] = ncd['plan'].str[:3]
ncd3 = ncd[ncd['plan'].isin(['APP_STD', 'ATT_STD', 'FIN_STD', 'PAY_STD'])]
ncd3 = ncd3.sort_values(by = ['member_id','calc_ymd','plan_prefix','paid_여부'])
ncd3_reduced = ncd3[['member_id', 'calc_ymd', 'plan', 'plan_prefix', 'paid_여부']].drop_duplicates(subset=['member_id', 'calc_ymd', 'plan_prefix'], keep = 'last')
paid_user_plans_wp = ncd3_reduced[ncd3_reduced['paid_여부'] == 'Y'].groupby(['member_id', 'calc_ymd'])['plan'].apply(lambda x: ', '.join(x)).reset_index(name='도메인WORKPLACE유료구독상세상품')
ncd = pd.merge(ncd, paid_user_plans_wp, on=['member_id', 'calc_ymd'], how='left')
paid_user_counts_wp = ncd3_reduced[ncd3_reduced['paid_여부'] == 'Y'].groupby(['member_id', 'calc_ymd']).size().reset_index(name='도메인WORKPLACE유료구독상세상품개수')
ncd = pd.merge(ncd, paid_user_counts_wp, on=['member_id', 'calc_ymd'], how='left')
ncd = ncd.drop(['plan_prefix'], axis=1)

In [9]:
ncd['plan_prefix'] = ncd['plan'].str[:3]
ncd4 = ncd[ncd['plan'].isin(['APP_T', 'ATT_T', 'FIN_T', 'PAY_T', 'APP_STD', 'ATT_STD', 'FIN_STD', 'PAY_STD'])]
ncd4 = ncd4.sort_values(by = ['member_id','calc_ymd','plan_prefix','paid_여부'])
ncd4_reduced = ncd4[['member_id', 'calc_ymd', 'plan', 'plan_prefix', 'paid_여부']].drop_duplicates(subset=['member_id', 'calc_ymd', 'plan_prefix'], keep = 'last')
user_plans_wp = ncd4_reduced.groupby(['member_id', 'calc_ymd'])['plan'].apply(lambda x: ', '.join(x)).reset_index(name='도메인WORKPLACE구독상세상품')
ncd = pd.merge(ncd, user_plans_wp, on=['member_id', 'calc_ymd'], how='left')
user_counts_wp = ncd4_reduced.groupby(['member_id', 'calc_ymd']).size().reset_index(name='도메인WORKPLACE구독상세상품개수')
ncd = pd.merge(ncd, user_counts_wp, on=['member_id', 'calc_ymd'], how='left')
ncd = ncd.drop(['plan_prefix'], axis=1)

In [10]:
def change_product_name1(product_list):
    if pd.isna(product_list):
        return np.nan
    
    products = product_list.split(', ')
    updated_products = []
    for product in products:
        if product.startswith('COR'):
            updated_products.append('웍스코어')
        elif product.startswith('DRV'):
            updated_products.append('드라이브')
        elif product.startswith('APP'):
            updated_products.append('경영지원')
        elif product.startswith('CNT'):
            updated_products.append('클로바노트')
        else:
            updated_products.append(product)
    return ', '.join(updated_products)

ncd['도메인유료구독상품_구분3'] = ncd['도메인유료구독상품'].apply(change_product_name1)
ncd['도메인구독상품_구분3'] = ncd['도메인구독상품'].apply(change_product_name1)

In [11]:
def change_product_name2(product_list):
    if pd.isna(product_list):
        return np.nan
    
    products = product_list.split(', ')
    updated_products = []
    for product in products:
        if product.startswith('APP'):
            updated_products.append('웍스 결재')
        elif product.startswith('ATT'):
            updated_products.append('웍스 근태')
        elif product.startswith('FIN'):
            updated_products.append('웍스 재무')
        elif product.startswith('PAY'):
            updated_products.append('웍스 급여')
        else:
            updated_products.append(product)
    return ', '.join(updated_products)

ncd['도메인WORKPLACE유료구독상세상품_구분3'] = ncd['도메인WORKPLACE유료구독상세상품'].apply(change_product_name2)
ncd['도메인WORKPLACE구독상세상품_구분3'] = ncd['도메인WORKPLACE구독상세상품'].apply(change_product_name2)

In [12]:
ncd['이전상품'] = ''
ncd['구분3'] = '웍스코어'
ncd.loc[ncd['plan'].str[0:3] == 'DRV','구분3'] = '드라이브'
ncd.loc[ncd['plan'].str[0:3] == 'CNT','구분3'] = '클로바노트'
ncd.loc[ncd['plan'].str[0:3].isin(['APP','ATT','FIN','PAY']),'구분3'] = '경영지원'
ncd = ncd.sort_values(by = ['member_id','calc_ymd','구분3','paid_여부'])

In [13]:
ncd = ncd.sort_values(by = ['회원번호','구분3','calc_ymd'])
ncd_drop = ncd.drop_duplicates(subset = ['회원번호','구분3'], keep = 'first').copy()
ncd_drop['plan_regist'] = ncd_drop['regist_ymd']
ncd_drop = ncd_drop[['회원번호','구분3','plan_regist']]
ncd = pd.merge(ncd, ncd_drop, how = 'left',on= ['회원번호','구분3'])

In [14]:
ncd.loc[ncd['paid_여부'] == 'N', '구분3'] = ncd['구분3'] + 'T'

In [15]:
ncd = ncd.sort_values(by = ['회원번호','구분3','calc_ymd'])
ncd_first = ncd.drop_duplicates(subset = ['회원번호','구분3'], keep = 'first').copy()

for member in ncd_first['회원번호'].unique():
    member_data = ncd_first[ncd_first['회원번호'] == member].sort_values(by='plan_regist').reset_index()
    
    for i in range(len(member_data)):
        current_date = member_data.loc[i, 'plan_regist']
        current_index = member_data.loc[i, 'index']
        
        previous_plans = member_data[member_data['plan_regist'] < current_date]
        
        if not previous_plans.empty:
            last_date = previous_plans['plan_regist'].max()
            last_plans = previous_plans[previous_plans['regist_ymd'] == last_date]['구분3'].tolist()
            
            unique_last_plans = list(dict.fromkeys(last_plans))
            if len(unique_last_plans) > 1:
                ncd_first.at[current_index, '이전상품'] = ', '.join(unique_last_plans)
            else:
                ncd_first.at[current_index, '이전상품'] = unique_last_plans[0]
        
        same_day_plans = member_data[member_data['plan_regist'] == current_date]['구분3'].str[0:4].unique()
        if len(same_day_plans) > 1:
            ncd_first.loc[member_data[member_data['plan_regist'] == current_date]['index'], '이전상품'] = '동시구독'


In [16]:
ncd = ncd.sort_values(by=['회원번호', '구분3', 'calc_ymd'])
ncd_first2 = ncd[ncd['paid_여부'] == 'Y'].drop_duplicates(subset=['회원번호', '구분3'], keep='first').copy()

for member in ncd_first2['회원번호'].unique():
    member_data = ncd_first2[ncd_first2['회원번호'] == member].sort_values(by='plan_regist').reset_index()
    
    for i in range(len(member_data)):
        current_date = member_data.loc[i, 'plan_regist']
        current_index = member_data.loc[i, 'index']
        
        previous_plans = member_data[member_data['plan_regist'] < current_date]
        
        if not previous_plans.empty:
            last_date = previous_plans['plan_regist'].max()
            last_plans = previous_plans[previous_plans['plan_regist'] == last_date]['구분3'].tolist()
            
            unique_last_plans = list(dict.fromkeys(last_plans))
            if len(unique_last_plans) > 1:
                ncd_first2.at[current_index, '이전상품'] = ', '.join(unique_last_plans)
            else:
                ncd_first2.at[current_index, '이전상품'] = unique_last_plans[0]
        
        same_day_plans = member_data[member_data['plan_regist'] == current_date]['구분3'].str[0:4].unique()
        if len(same_day_plans) > 1:
            ncd_first2.loc[member_data[member_data['plan_regist'] == current_date]['index'], '이전상품'] = '동시구독'


In [17]:
ncd = pd.merge(ncd, ncd_first[['회원번호','구분3','이전상품']],how='left',on = ['회원번호','구분3'])
ncd = pd.merge(ncd, ncd_first2[['회원번호', '구분3', '이전상품']], how='left', on=['회원번호', '구분3'])

In [18]:
ncd = ncd.drop(['이전상품_x','구분3','plan_regist'],axis = 1)
ncd = ncd.rename(columns={'이전상품_y': '이전상품_Trial포함','이전상품':'Paid_이전상품'})

In [19]:
def select_shortest_unique_product(products):
    if pd.isna(products):
        return ''
    product_list = str(products).split(', ')
    unique_products = {}

    for product in product_list:
        prefix = product[:4]
        if prefix not in unique_products or len(product) < len(unique_products[prefix]):
            unique_products[prefix] = product

    return ', '.join(unique_products.values())

ncd['이전상품_Trial포함'] = ncd['이전상품_Trial포함'].apply(select_shortest_unique_product)
ncd['Paid_이전상품'] = ncd['Paid_이전상품'].apply(select_shortest_unique_product)

In [20]:
#매출정보컬럼추가
boa1 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202201)new.xlsx')
boa2 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202202)new.xlsx')
boa3 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202203)new.xlsx')
boa4 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202204)new.xlsx')
boa5 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202205)new.xlsx')
boa6 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202206)new.xlsx')
boa7 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202207)new.xlsx')
boa8 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202208)new.xlsx')
boa9 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202209)new.xlsx')
boa10 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202210)new.xlsx')
boa11 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202211)new.xlsx')
boa12 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202212)new.xlsx')
boa13 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202301)new.xlsx')
boa14 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202302)new.xlsx')
boa15 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202303)new.xlsx')
boa16 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202304)new.xlsx')
boa17 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202305)new.xlsx')
boa18 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202306)new.xlsx')
boa19 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202307)new.xlsx')
boa20 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202308)new.xlsx')
boa21 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202309)new.xlsx')
boa22 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202310)new.xlsx')
boa23 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202311)new.xlsx')
boa24 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202312)new.xlsx')
boa25 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202401)new.xlsx')
boa26 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202402)new.xlsx')
boa27 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202403)new.xlsx')
boa28 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202404)new.xlsx')
boa29 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202405)new.xlsx')
boa30 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202406)new.xlsx')
boa31 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202407)new.xlsx')
boa32 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202408)new.xlsx')
boa33 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202409)new.xlsx')
boa34 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202410)new.xlsx')
boa35 = pd.read_excel('/Users/user/Documents/진행중 업무/데이터/BOA/BOA(202411)new.xlsx')
boa = pd.concat([boa1, boa2, boa3, boa4, boa5, boa6, boa7, boa8, boa9, boa10,
                 boa11, boa12, boa13, boa14, boa15, boa16, boa17, boa18, boa19, boa20, 
                 boa21, boa22, boa23, boa24, boa25, boa26, boa27, boa28, boa29, boa30,
                 boa31, boa32, boa33, boa34, boa35])
boa.rename(columns = {'NCP회원번호':'회원번호','상품구분':'구분3'}, inplace=True)
boa = boa[boa['채널'] == '파트너 매출']
boa = boa[boa['매출금액']>0]
boa['이용금액대비매출금액'] = boa['매출금액']/boa['이용금액']
pt = boa.groupby(['회원번호','구분3'])['이용금액대비매출금액'].mean().reset_index()

In [21]:
ncd['month_info'] = ncd['미터링일시'].astype(str).str[0:6]
ncd['dayCount'] = ncd.groupby(['회원번호', '미터링유형코드명','month_info'])['회원번호'].transform('size')
ncd['약정할인금액'] = np.where(
        (ncd['plan'].isin(['COR_STD_P'])) & (ncd['약정여부'] == 'Y'), 
        2000, 
        np.where(
            (ncd['plan'].isin(['COR_LT','COR_STD','DRV_LT','DRV_STD','DRV_STD_P'])) & (ncd['약정여부'] == 'Y'), 
            1000,
            np.where(
                (ncd['plan'].isin(['PAY_STD'])) & (ncd['약정여부'] == 'Y'), 
                1500,
                np.where(
                    (ncd['plan'].isin(['APP_STD'])) & (ncd['약정여부'] == 'Y'), 
                    1000, 
                    np.where(
                        (ncd['plan'].isin(['ATT_STD','FIN_STD'])) & (ncd['약정여부'] == 'Y'), 
                        50,
                        np.where(
                            (ncd['plan'].isin(['COR_LT','COR_STD','COR_STD_P','DRV_LT','DRV_STD','DRV_STD_P','APP_STD','ATT_STD','FIN_STD','PAY_STD'])) & (ncd['약정여부'] == 'N'), 
                            0,
                            0
                        )
                    )
                )
            )
        )
    )
ncd['미터링일시'] = ncd['미터링일시'].astype(str)
ncd['월별일수'] = np.where(
    ncd['미터링일시'].str[4:6].isin(['01', '03', '05', '07', '08', '10', '12']),
    31,
    np.where(
        ncd['미터링일시'].str[4:6].isin(['04', '06', '09', '11']),
        30,
        np.where(
            (ncd['미터링일시'].str[0:4].astype(int) % 4 == 0),
            29, 
            28
        )
    )
)
ncd['미터링일시'] = ncd['미터링일시'].astype(int)
ncd['일요금(당일)'] = (ncd['요금']/ncd['월별일수'])*ncd['사용량']
ncd['약정할인 후 요금'] = ncd['요금'] - ncd['약정할인금액']
ncd['약정할인율'] = ncd['약정할인금액']/ncd['요금']
ncd['결합할인 여부'] = 'Y'
ncd.loc[ncd['plan'].isin(['ATT_STD', 'FIN_STD', 'PAY_STD']), '결합할인 여부'] = 'N'
ncd['결합할인 여부'] = np.where(
    (ncd.duplicated(subset=['회원번호','미터링일시','paid_여부','약정여부','결합할인 여부'], keep=False)) & (ncd['paid_여부'] == 'Y') & (ncd['약정여부'] == 'Y') & (ncd['결합할인 여부'] == 'Y'),
    'Y',
    'N')
ncd.loc[ncd['약정여부'] == 'N', '결합할인 여부'] = 'N'
ncd['미터링일시'] = ncd['미터링일시'].astype(str)
ncd['일매출(당일)'] = np.where(
    (ncd['결합할인 여부'] == 'Y'),
    ((ncd['요금']/ncd['월별일수'])*ncd['사용량']*(1-ncd['약정할인율']))*0.9,
    (ncd['요금']/ncd['월별일수'])*ncd['사용량']*(1-ncd['약정할인율']))

ncd['구분3'] = ''
ncd.loc[ncd['plan'].str[0:3]=='COR','구분3']='NAVER WORKS'
ncd.loc[ncd['plan'].str[0:3]=='DRV','구분3']='WORKS DRIVE'
ncd.loc[ncd['plan'].str[0:3]=='CNT','구분3']='클로바노트'
ncd.loc[ncd['plan'].str[0:3].isin(['APP','ATT','FIN','PAY']),'구분3']='WORKPLACE'

ncd['일매출(당일)'] = ncd['일매출(당일)'].fillna(0)
ncd = pd.merge(ncd, pt, how='left', on=['회원번호', '구분3'])
ncd.loc[~ncd['이용금액대비매출금액'].isna(), '일매출(당일)'] = ncd['일요금(당일)'] * ncd['이용금액대비매출금액']
ncd = ncd.drop(['month_info','dayCount','약정할인금액','월별일수','약정할인 후 요금','약정할인율','결합할인 여부','구분3','이용금액대비매출금액'], axis = 1)

In [22]:
#ncd_new = ncd[ncd['regist_ymd']>20241005]
#ncd_new_t = ncd_new[ncd_new['paid_여부']=='N']
#ncd_new_t_20241105 = ncd_new_t[ncd_new_t['미터링일시']==20241105]
#ncd_new_t_no_user_20241105 = ncd_new_t_20241105[ncd_new_t_20241105['사용량']==0]
#ncd_new_t_no_user_20241105.to_csv('/Users/user/Documents/진행중 업무/new_domain_trial_no_user.csv', index=False)

In [23]:
def format_date(x):
    try:
        x = int(float(x))
        year = str(x)[:4]
        month = str(int(str(x)[4:6]))
        day = str(int(str(x)[6:]))
        return f"{year}.{month}.{day}"
    except (ValueError, TypeError):
        return None 
        
def preprocess4 (df) :
    df['calc_ymd'] = df['calc_ymd'].apply(format_date)
    df['regist_ymd'] = df['regist_ymd'].apply(format_date)
    df['delete_ymd'] = df['delete_ymd'].apply(format_date)
    df['domainRegistdate'] = df['domainRegistdate'].apply(format_date)
    df['전환일'] = df['전환일'].apply(format_date)

    df['전환여부'] = np.where(df['paid_여부']== 'Y', np.nan, df['전환여부'])
    df['전환일'] = np.where(df['paid_여부']== 'Y', np.nan, df['전환일'])

    return df

ncd = preprocess4(ncd)

In [24]:
ncd.to_csv('/Users/user/Documents/진행중 업무/ncd_metering2.csv',index=False)

In [3]:
ncd = pd.read_csv('/Users/user/Documents/진행중 업무/ncd_metering2.csv')

/var/folders/yj/y3b7nzyn7zv0l9bkqtj_37g40000gn/T/ipykernel_17738/4125878327.py:1: DtypeWarning: Columns (31,35,36,41,43,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  ncd = pd.read_csv('/Users/user/Documents/진행중 업무/ncd_metering2.csv')


In [4]:
ncd1231 = ncd[ncd['미터링일시']==20241231]

In [7]:
ncd1231[ncd1231['plan']=='COR_LT']['사용량'].sum()

54876.0

In [9]:
ncd1231[ncd1231['plan']=='COR_STD']['사용량'].sum()

435259.0

In [11]:
ncd1231[ncd1231['plan']=='COR_STD_P']['사용량'].sum()

81350.0

In [13]:
ncd1231[ncd1231['plan']=='DRV_LT']['사용량'].sum()

1324.0

In [15]:
ncd1231[ncd1231['plan']=='DRV_STD']['사용량'].sum()

843.0

In [17]:
ncd1231[ncd1231['plan']=='DRV_STD_P']['사용량'].sum()

2764.0